In [1]:
from datetime import datetime
import config, input_tables
from steps import psfsubtraction,klipphotometry
from stralog import getLogger
import os
import pkg_resources as pkg

First, we need to initialize the logger here.

In [3]:
if 'SHARED_LOG_FILE' not in os.environ:
    os.environ['SHARED_LOG_FILE'] = f'straklip_{datetime.now().strftime("%Y-%m-%d_%H%M")}.log'

getLogger('straklip', setup=True, logfile=os.environ['SHARED_LOG_FILE'],debu=False,
          configfile=pkg.resource_filename('straklip', './config/logging.yaml'))

<Logger straklip (DEBUG)>

For this tutorial we will start form the previously generated dataframes from the pipeline.
Note that in the pipe.yaml, we use unq_ids_list: [52] so the pipeline will run the reduction only on this target, to showcase the pipelien and save time and space.

In [4]:
pipe_cfg='/Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/pipe.yaml' #or where these files are
data_cfg='/Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/data.yaml'
pipe_cfg = config.configure_pipeline(pipe_cfg,pipe_cfg=pipe_cfg,data_cfg=data_cfg,dt_string=datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
data_cfg = config.configure_data(data_cfg,pipe_cfg)

2025-06-05 12:39:42 config                      :INFO     (configure_pipeline:72[pid=48146]) 
StraKLIP pipeline started at date and time: 05/06/2025 12:39:42
Pipe_cfg: /Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/pipe.yaml
Data_cfg: /Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/data.yaml

2025-06-05 12:39:42 config                      :INFO     (configure_data:153[pid=48146]) Validation of default labels and data successful!


Once the "pipe_cfg" and the "data_cfg" are configuration, we can load pre-existing dataframes.

In this case we use "skip_originals" True and "load" True to tell the pipeline tom not look for inpout catalogs, but for the dataframe already generated by the pipeline.

NOTE: for this tutorial, we are forcing the pipeline to work on only the "unq_id" 52 to speed up the process, as follow (you can achieve the same result by changing the "unq_ids_list" option in the "pipe.yaml"):

In [5]:
pipe_cfg.unq_ids_list = [52]

In [6]:
dataset = input_tables.Tables(data_cfg, pipe_cfg, skip_originals=True)
DF = config.configure_dataframe(dataset,load=True)

2025-06-05 12:39:46 config                      :WARNING  (configure_dataframe:226[pid=48146]) get_Av_dict currently only supports VEGAmag system. Please provide your own set of AVs if in a differest system as AVs : {ext: {mag_filter : value}} in the data.yaml under target
2025-06-05 12:39:46 ancillary                   :INFO     (get_Av_dict:425[pid=48146]) before dust, V =  0.0 mag(VEGA)
2025-06-05 12:39:46 ancillary                   :INFO     (get_Av_dict:426[pid=48146]) after dust, V = 1.0146 mag(VEGA)
2025-06-05 12:39:46 ancillary                   :INFO     (get_Av_dict:445[pid=48146]) Av = 1.0146 mag


2025-06-05 12:39:47 ancillary                   :INFO     (get_Av_dict:489[pid=48146]) AV=0 wfc3,uvis2,f814w 0.0 mag(VEGA)
2025-06-05 12:39:47 ancillary                   :INFO     (get_Av_dict:490[pid=48146]) AV=1 wfc3,uvis2,f814w 0.6094 mag
2025-06-05 12:39:47 ancillary                   :INFO     (get_Av_dict:489[pid=48146]) AV=0 wfc3,uvis2,f850lp 0.0 mag(VEGA)
2025-06-05 12:39:47 ancillary                   :INFO     (get_Av_dict:490[pid=48146]) AV=1 wfc3,uvis2,f850lp 0.4694 mag
2025-06-05 12:39:47 config                      :INFO     (configure_dataframe:272[pid=48146]) Fetching dataframes from /Users/gstrampelli/StraKLIP/Tutorial/out


In [ ]:
DF.keys

['unq_targets',
 'crossmatch_ids',
 'mvs_targets',
 'mvs_candidates',
 'unq_candidates']

The next step wil perform PSF subtraction for each target in the "mvs_targets_df", populating both the "mvs_candidates_df" and the "unq_candidates_df, creating a "residual_tiles" folder in the "out" directory with visual summary of the subtraction and updating the tile cube in the "mvs_tiles" and "median_tiles" with the outcome of the subtraction: a.k.a. the residuals for each KLIP mode and the models.

In [8]:
psfsubtraction.run({'DF': DF, 'dataset': dataset})

2025-06-05 12:40:00 steps.psfsubtraction        :INFO     (run:237[pid=48146]) Performing KLIP PSF subtraction on tiles.
2025-06-05 12:40:00 straklip.config             :INFO     (make_paths:117[pid=48146]) "/Users/gstrampelli/StraKLIP/Tutorial/out/residual_tiles/f814w" exists, and will not be created.
2025-06-05 12:40:00 ancillary                   :INFO     (parallelization_package:969[pid=48146]) Max allowable workers 1, # of elements 1 , # of chunk 1 approx # of elemtent per chunks 1 (chunksize)


100%|██████████| 1/1 [00:00<00:00, 26.22it/s]


2025-06-05 12:40:08 straklip.config             :INFO     (make_paths:114[pid=48146]) Creating "/Users/gstrampelli/StraKLIP/Tutorial/out/residual_tiles/f850lp"
2025-06-05 12:40:08 ancillary                   :INFO     (parallelization_package:969[pid=48146]) Max allowable workers 1, # of elements 1 , # of chunk 1 approx # of elemtent per chunks 1 (chunksize)


100%|██████████| 1/1 [00:00<00:00,  5.02it/s]


2025-06-05 12:40:15 dataframe                   :INFO     (save_dataframes:109[pid=48146]) Saving the the following keys in ['unq_targets_df', 'crossmatch_ids_df', 'mvs_targets_df', 'mvs_candidates_df', 'unq_candidates_df'] to .csv files in /Users/gstrampelli/StraKLIP/Tutorial/out


After generating the tiles for the residuals, we run the aperture photometry on each of them, to gather some basic information for the upcoming analysis. This step provide a basic aperture photometry and candidate identification (locating the brightest pixel in the residuals that persist across different filters and/or KLIP modes as requested).

In [11]:
klipphotometry.run({'DF': DF, 'dataset': dataset})

2025-06-05 12:42:12 steps.klipphotometry        :INFO     (update_candidate_dataframe:1184[pid=48146]) Updating the candidates dataframe
2025-06-05 12:42:12 steps.klipphotometry        :INFO     (update_companion_ZPT:885[pid=48146]) Working on the zero points for candidates
2025-06-05 12:42:12 ancillary                   :INFO     (parallelization_package:969[pid=48146]) Max allowable workers 8, # of elements 57 , # of chunk 19 approx # of elemtent per chunks 3 (chunksize)
2025-06-05 12:42:16 steps.klipphotometry        :INFO     (update_candidates:996[pid=48146]) Working on the candidates
2025-06-05 12:42:16 ancillary                   :INFO     (parallelization_package:969[pid=48146]) Max allowable workers 1, # of elements 1 , # of chunk 1 approx # of elemtent per chunks 1 (chunksize)
2025-06-05 12:42:18 steps.klipphotometry        :INFO     (update_candidate_dataframe:1203[pid=48146]) Updating the median tiles for the candidates
2025-06-05 12:42:18 steps.klipphotometry        :INFO 

In [12]:
DF.mvs_candidates_df

,mvs_ids,x_tile_f814w,x_tile_f850lp,y_tile_f814w,y_tile_f850lp,x_rot_f814w,x_rot_f850lp,y_rot_f814w,y_rot_f850lp,counts_f814w,...,tp_above_th_f814w,tp_above_th_f850lp,tp_above_nsigma_f814w,tp_above_nsigma_f850lp,fp_above_th_f814w,fp_above_th_f850lp,fp_above_nsigma_f814w,fp_above_nsigma_f850lp,auc_f814w,auc_f850lp
0,52,17.0,18.0,21.0,22.0,18.785,20.195,22.919,22.822,4507.388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
DF.unq_candidates_df


,unq_ids,mass,emass,sep,mkmode,n_f814w,n_f850lp,nsigma_f814w,nsigma_f850lp,m_f814w,...,tp_above_th_f814w,tp_above_th_f850lp,tp_above_nsigma_f814w,tp_above_nsigma_f850lp,fp_above_th_f814w,fp_above_th_f850lp,fp_above_nsigma_f814w,fp_above_nsigma_f850lp,auc_f814w,auc_f850lp
0,52,NaN,NaN,2.995,7.0,1.0,1.0,122.839277,71.885221,22.614,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Once the basic photometry has been performed on the candidates, we can move on on the final stage of the pipeline and perform some more advanced analysis.